# interp_smoothed_strat
Read stratification from "lowpass" smoothed strat netCDF file and interpolate it on desired grid (in space and time)

this was to diagnoze $\delta z$ at the base of the mixed layer, as $\delta z = \frac{\delta b}{<N^2>}$ ($\delta b = b - <b>$)

In [16]:
% matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
from netCDF4 import Dataset
import sys
import time as tm
for pap in ['/home/lahaye/Coding/Py3_ROMS_Modules']:
    if pap not in sys.path:
        sys.path.append(pap)
import R_tools_fort as toolsf
from R_smooth import get_tri_coef
from scipy.spatial import Delaunay
import scipy.interpolate as itp

KRYPTON = "/data0/project/vortex/lahaye/"
RUCHBA = "/net/ruchba/local/tmp/2/lahaye/"

In [6]:
# some parameters

imin, imax = 200, 400
jmin, jmax = 800, 1000
it = 10     # time index (original or subsampled time series)

path_tser = KRYPTON + "luckyt_tseries/luckyt_tseries_{}.nc" #.format(var_iz{iz})
path_strat = RUCHBA + "luckyt_tseries_lf/luckyt_subsamp_strat_win.nc"
path_grid = KRYPTON + "lucky_corgrd.nc"

In [20]:
# outside the routine: load x-, y-grid, determine hor. interpolating coeff (delaunay triangulation)

# load "time series" netCDF file and get hor. indices arrays

nc = Dataset(path_tser.format("2Dvars"))
xi = nc.variables['xi_rho'][:].astype(int)
indx, = np.where((xi>=imin) & (xi<=imax))
eta = nc.variables['eta_rho'][:].astype(int)
indy, = np.where((eta>=jmin) & (eta<=jmax))
xx = nc.variables['lon_rho'][indy,indx].T
yy = nc.variables['lat_rho'][indy,indx].T
Nx, Ny = xx.shape
xo, eto = xi[indx], eta[indy]
hc = nc.hc
Cs_r = nc.Cs_r
Cs_w = nc.Cs_w
timef = nc.variables['time'][:]
nc.close()
Nz = Cs_r.size

# load topography to compute zlevs
nc = Dataset(path_grid)
topo = nc.variables['h'][eta[indy],xi[indx]].T
nc.close()

zr, zw = toolsf.zlevs(topo,np.zeros(topo.shape),hc,Cs_r,Cs_w)

# load grid in strat file
nc = Dataset(path_strat)
xi = nc.variables['xi_rho'][:].astype(int)
i1 = np.where(xi>imin)[0][0]-1
i2 = np.where(xi<imax)[0][-1]+2
eta = nc.variables['eta_rho'][:].astype(int)
j1 = np.where(eta>jmin)[0][0]-1
j2 = np.where(eta<jmax)[0][-1]+2
lon = nc.variables['lon_rho'][j1:j2,i1:i2].T
lat = nc.variables['lat_rho'][j1:j2,i1:i2].T
times = nc.variables['time'][:]
nc.close()

nc = Dataset(path_grid)
h = nc.variables['h'][eta[j1:j2],xi[i1:i2]].T
nc.close()
_, zz = toolsf.zlevs(h,np.zeros(h.shape),hc,Cs_r,Cs_w)
nx, ny = h.shape
xi, eta = xi[i1:i2], eta[j1:j2]
zref= zz[0,0,1:-1]/h[0,0]   # for vertical integration
print('just finished loading stuffs')

# bilinear interpolation using delaunay triangulation
tmes, tmeb = tm.clock(), tm.time()
elem, coef = get_tri_coef(lon,lat,xx,yy)
print("triangulation",tm.clock()-tmes,tm.time()-tmeb)

just finished loading stuffs
triangulation 0.9951540000000003 0.9988946914672852


In [24]:
# in function: interpolate
nc = Dataset(path_strat, "r")
data = nc.variables['bvf_lowf'][:,:,j1:j2,i1:i2]    # t, z
nc.close()

# z-interpolation
finterp = []
tmes, tmeb = tm.clock(), tm.time()
for ii in range(nx):
    for jj in range(ny):
        finterp.append(itp.RectBivariateSpline(\
                times,zz[ii,jj,:],np.pad(data[:,:,jj,ii],((0,0),(1,1)),'edge')))
finterp = np.array(finterp)
print("interpolated",tm.clock()-tmes,tm.time()-tmeb)

tmes, tmeb = tm.clock(), tm.time()
datint = np.zeros((Nz-1,Ny,Nx))
for ii in range(Nx):
    for jj in range(Ny):
        valitp = np.array([finterp[kk](times[it],zw[ii,jj,1:-1]).squeeze() \
                            for kk in elem[ii,jj,:]])
        datint[:,jj,ii] = (coef[ii,jj,:,None]*valitp).sum(axis=0)

print("reconstructed",tm.clock()-tmes,tm.time()-tmeb)

interpolated 0.2625840000000004 0.2626383304595947
reconstructed 0.5175140000000003 0.5193440914154053


In [28]:
data.shape

(16, 79, 35, 35)